In [2]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
from tensorflow.python.ops import rnn, rnn_cell
import random

# Hyperparameters

In [3]:
train_length = 1000
test_length = 1500

discount_rate = 0.995
rnn_size = 10

sampling_rate = 200
striding = 100
train_steps = 1000

In [4]:
train_input_data = []
train_output_data = [] 

In [5]:
input_file_name1 = 'rnn_v6_input_average.csv'
input_file_name2 = 'rnn_v6_input_variation.csv'
input_file_name3 = 'rnn_v6_output.csv'

input_file = open(input_file_name1,'r',newline='')
reader = csv.reader(input_file, delimiter=',')
temp = []
for row in reader:
    temp.append(row)
train_input_data.append(temp)

input_file = open(input_file_name2,'r',newline='')
reader = csv.reader(input_file, delimiter=',')
temp = []
for row in reader:
    temp.append(row)
train_input_data.append(temp)

input_file = open(input_file_name3,'r',newline='')
reader = csv.reader(input_file, delimiter=',')
for row in reader:
    train_output_data.append(row)

In [6]:
train_input_data =np.array(train_input_data, dtype=np.float32)
train_output_data=np.array(train_output_data, dtype=np.float32)
print(train_input_data.shape)
print(train_output_data.shape)

(2, 1000, 129)
(996, 129)


In [ ]:
train_input_data = np.transpose(train_input_data, [2,0,1])
train_output_data = np.transpose(train_ouput_data,[1,0])
print(train_input_data.shape)
print(train_output_data.shape)

# Neural network

In [24]:
x = tf.placeholder(tf.float32, shape=[None, 2, train_length])
y = tf.placeholder(tf.float32, shape=[None, train_length])
test_x = tf.placeholder(tf.float32, shape=[None, 2, test_length])

rnn_tei_x = tf.transpose(test_x, [2,0,1])
rnn_tei_x = tf.reshape(rnn_tei_x, [-1,2])
rnn_tei_x = tf.split(0, test_length, rnn_tei_x)

rnn_tri_x = tf.transpose(x, [2,0,1])
rnn_tri_x = tf.reshape(rnn_tri_x, [-1,2])
rnn_tri_x = tf.split(0, train_length, rnn_tri_x)

with tf.variable_scope("rnn") as scope:
    lstm_cell = rnn_cell.LSTMCell(rnn_size)

    W = tf.Variable(tf.truncated_normal([rnn_size,1], stddev=0.01))
    b = tf.Variable(tf.constant(0.01, shape=[1]))
    
    outputs, states = rnn.rnn(lstm_cell, inputs = rnn_tri_x, dtype=tf.float32)
    temp_outputs = tf.pack(outputs)
    temp_outputs = tf.transpose(temp_outputs, [1,0,2])
    temp_outputs = tf.reshape(temp_outputs,[-1,rnn_size])
    temp_outputs = tf.matmul(temp_outputs,W)+b
    temp_outputs = tf.reshape(temp_outputs,[-1,train_length,1])
    print(temp_outputs)
    
    scope.reuse_variables()
    test_outputs, test_states = rnn.rnn(lstm_cell, inputs = rnn_tei_x, dtype =tf.float32)
    temp_test_outputs = tf.pack(test_outputs)
    temp_test_outputs = tf.transpose(temp_test_outputs, [1,0,2])
    temp_test_outputs = tf.reshape(temp_test_outputs,[-1,rnn_size])
    temp_test_outputs = tf.matmul(temp_test_outputs,W)+b
    temp_test_outputs = tf.reshape(temp_test_outputs,[-1,test_length,1])    
    print(temp_test_outputs)
    
    temp_y = tf.reshape(y,[-1,train_length,1])
    print(temp_y)

Tensor("rnn_9/pack:0", shape=(1000, ?, 10), dtype=float32)
Tensor("rnn_9/transpose:0", shape=(?, 1000, 10), dtype=float32)
Tensor("rnn_9/add:0", shape=(?, 1), dtype=float32)
Tensor("rnn_9/Reshape_1:0", shape=(?, 1000, 1), dtype=float32)
Tensor("rnn_9/Reshape_3:0", shape=(?, 1500, 1), dtype=float32)
Tensor("rnn_9/Reshape_4:0", shape=(?, 1000, 1), dtype=float32)


In [29]:
cost = tf.reduce_mean(tf.square(temp_outputs-temp_y))
train = tf.train.AdamOptimizer(1e-3).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [31]:
for step in range(train_steps+1):
    opt, c = sess.run([train, cost], feed_dict={x: train_input_data, y: train_output_data})
    if step%100==0:
        print("{}th run cost = {}".format(step,c))

0th run cost = 0.11382706463336945
100th run cost = 0.05200079083442688
200th run cost = 0.04408244416117668
300th run cost = 0.03964800015091896
400th run cost = 0.03550788760185242
500th run cost = 0.027448279783129692
600th run cost = 0.021361328661441803


KeyboardInterrupt: 

train_data_set = []
temp_train_data_set = [1,6,8,9,10,11,12,14,15,16,17,18]
train_data_set.append(temp_train_data_set)
temp_train_data_set = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,20,21]
train_data_set.append(temp_train_data_set)
temp_train_data_set = [1,8,9,10,16,17]
train_data_set.append(temp_train_data_set)
print(train_data_set)

In [23]:
tf.add_to_collection('test_input', test_x)
tf.add_to_collection('hypothesis', temp_test_outputs)
saver = tf.train.Saver()
saver.save(sess,'rnn_v6')

'rnn_v5'